In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [5]:

# Load the data
data_path = "../data/Eye-tracking Output/cleaned_data.csv"
df = pd.read_csv(data_path)


In [9]:
## Normalizing data ##

# Feature selection of relevant columns
relevant_columns = ['Time.s', 'Point of Regard Left X [px]', 'Point of Regard Left Y [px]',
                    'Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
                    'Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]', 'Tracking Ratio [%]',
                    'Category Left', 'Category Right', 'Participant', 'Trial', 'Stimulus', 'Gender', 'Age', 'Class', 'CARS Score']
df_relevant = df[relevant_columns]

# List of numerical columns to scale
numerical_columns = ['Time.s', 'Point of Regard Left X [px]', 'Point of Regard Left Y [px]',
                     'Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
                     'Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]', 'Tracking Ratio [%]']

# Convert columns to numeric, coercing errors to NaN
for col in numerical_columns:
    df_relevant[col] = pd.to_numeric(df_relevant[col], errors='coerce')

# Define a function to fill NaN with the mean of the previous and next row
def fill_with_row_mean(df, col):
    # First, forward fill the first NaN (if any)
    df[col] = df[col].fillna(method='ffill')
    
    # Then, fill the rest with the mean of the previous and next row
    df[col] = df[col].fillna((df[col].shift(1) + df[col].shift(-1)) / 2)
    
    return df[col]

# Apply this function to each numerical column
for col in numerical_columns:
    df_relevant[col] = fill_with_row_mean(df_relevant, col)

# Handle any remaining NaNs, especially at the end of the DataFrame
df_relevant.fillna(method='bfill', inplace=True)

# Normalize data per combination of Trial, Participant, and Stimulus
for (trial, participant, stimulus), group_data in df_relevant.groupby(['Trial', 'Participant', 'Stimulus']):
    scaler = MinMaxScaler()
    # Apply the scaler to all numerical columns for this group
    df_relevant.loc[group_data.index, numerical_columns] = scaler.fit_transform(group_data[numerical_columns])

# Save the normalized data
df_relevant.to_csv("../data/Eye-tracking Output/normalized_data.csv", index=False)

/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_90277/3250144389.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant[col] = pd.to_numeric(df_relevant[col], errors='coerce')
/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_90277/3250144389.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(method='ffill')
/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_90277/3250144389.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [12]:
# Splitting the data into training and test sets

def create_sequences(data, sequence_length):
    sequences = []
    labels = []  # Adjust based on your task
    for i in range(len(data) - sequence_length):
        sequence = data[i:i + sequence_length]
        label = ...  # Define your label
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequence_length = 60  # 1 second of data
data_sequences, labels = create_sequences(df_relevant.values, sequence_length)

# Reshape for LSTM
X = data_sequences.reshape((data_sequences.shape[0], sequence_length, data_sequences.shape[2]))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False)



In [ ]:

# Define the LSTM model
model = Sequential()
# Add LSTM layers and other necessary layers
# ...

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
# ...

# Evaluate the model
# ...

# Make predictions
# ...

NameError: name 'Sequential' is not defined